# Using Anthropic MCP from Python

This notebook shows how to call Claude Sonnet-4 **with multiple knowledge sources** via the new **MCP (Multi-Context Prompt)** parameter.

## Contents
1. Install prerequisites
2. Configure credentials
3. One-shot query with two MCP servers
4. Continuing the same conversation
5. Mixing structured and natural-language follow-ups

In [ ]:
# 1. Install prerequisites
!pip install -q anthropic rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 4.4 MB/s eta 0:00:00


## 2. Configure credentials
Replace `YOUR_ANTHROPIC_API_KEY` with your own key (get one from <https://console.anthropic.com>).  
Storing secrets in environment variables keeps them out of version control.

In [ ]:
import os
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"

## 3. One-shot query with two MCP servers
Below we target **two** external data sources:
1. An historical stock-price feed (`mcp_api_apibricks_io`)
2. The SEC filings stream (`mcp_sec_apibricks_io`)

Both are indexed the same way you would list a single server—just pass a list of dicts to `mcp_servers`.

In [ ]:
from anthropic import Anthropic
from rich import print

client = Anthropic()

api_key = "YOUR-FINFEED-API-KEY"

MCP_SERVERS = [
    {
        "type": "url",
        "url": "https://mcp-historical.stock.finfeedapi.com/mcp",
        "name": "mcp_api_apibricks_io",
        "authorization_token": api_key
    },
    {
        "type": "url",
        "url": "https://mcp.sec.finfeedapi.com/sse",
        "name": "mcp_sec_apibricks_io",
        "authorization_token": api_key
    }
]

response = client.beta.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1000,
    messages=[{
        "role": "user",
        "content": "When did Apple last file an 8-K report?"
    }],
    mcp_servers=MCP_SERVERS,
    extra_headers={"anthropic-beta": "mcp-client-2025-04-04"}
)

print(response.content)

[
    BetaTextBlock(
        citations=None,
        text="I'll help you find Apple's most recent 8-K filing. Let me search for their latest 8-K report.",
        type='text'
    ),
    BetaMCPToolUseBlock(
        id='mcptoolu_01Xbudgikg27TqurBgRUM9sm',
        input={
            'ticker': 'AAPL',
            'form_type': '8-K',
            'page_size': 5,
            'sort_by': 'FilingDate',
            'sort_order': 'desc'
        },
        name='Query_SEC_filing_metadata',
        server_name='mcp_sec_apibricks_io',
        type='mcp_tool_use'
    ),
    BetaMCPToolResultBlock(
        content=[
            BetaTextBlock(
                citations=None,
                text='[\n  {\n    "cik": 320193,\n    "accession_number": "0001140361-25-018400",\n    
"filing_date": "2025-05-12",\n    "report_date": "2025-05-05",\n    "acceptance_date_time": 
"2025-05-12T00:00:00.0000000Z",\n    "act": "34",\n    "form": "8-K",\n    "film_number": "25935352",\n    "items":
"8.01,9.01",\n    "size": 887109,\n    "is_xbrl": true,\n    "is_inline_xbrl": true,\n    "primary_document": 
"ef20048691_8k.htm",\n    "primary_doc_description": "8-K",\n    "source_file": 
"https://www.sec.gov/Archives/edgar/data/320193/000114036125018400/ef20048691_8k.htm"\n  },\n  {\n    "cik": 
320193,\n    "accession_number": "0000320193-25-000055",\n    "filing_date": "2025-05-01",\n    "report_date": 
"2025-05-01",\n    "acceptance_date_time": "2025-05-01T00:00:00.0000000Z",\n    "act": "34",\n    "form": "8-K",\n 
"film_number": "25903731",\n    "items": "2.02,9.01",\n    "size": 436093,\n    "is_xbrl": true,\n    
"is_inline_xbrl": true,\n    "primary_document": "aapl-20250501.htm",\n    "primary_doc_description": "8-K",\n    
"source_file": "https://www.sec.gov/Archives/edgar/data/320193/000032019325000055/aapl-20250501.htm"\n  },\n  {\n  
"cik": 320193,\n    "accession_number": "0001140361-25-005876",\n    "filing_date": "2025-02-25",\n    
"report_date": "2025-02-25",\n    "acceptance_date_time": "2025-02-25T00:00:00.0000000Z",\n    "act": "34",\n    
"form": "8-K",\n    "film_number": "25664289",\n    "items": "5.07",\n    "size": 291815,\n    "is_xbrl": true,\n  
"is_inline_xbrl": true,\n    "primary_document": "ef20044022_8k.htm",\n    "primary_doc_description": "8-K",\n    
"source_file": "https://www.sec.gov/Archives/edgar/data/320193/000114036125005876/ef20044022_8k.htm"\n  },\n  {\n  
"cik": 320193,\n    "accession_number": "0000320193-25-000007",\n    "filing_date": "2025-01-30",\n    
"report_date": "2025-01-30",\n    "acceptance_date_time": "2025-01-30T00:00:00.0000000Z",\n    "act": "34",\n    
"form": "8-K",\n    "film_number": "25574180",\n    "items": "2.02,9.01",\n    "size": 407697,\n    "is_xbrl": 
true,\n    "is_inline_xbrl": true,\n    "primary_document": "aapl-20250130.htm",\n    "primary_doc_description": 
"8-K",\n    "source_file": "https://www.sec.gov/Archives/edgar/data/320193/000032019325000007/aapl-20250130.htm"\n 
},\n  {\n    "cik": 320193,\n    "accession_number": "0001140361-25-000228",\n    "filing_date": "2025-01-03",\n   
"report_date": "2024-12-30",\n    "acceptance_date_time": "2025-01-03T00:00:00.0000000Z",\n    "act": "34",\n    
"form": "8-K",\n    "film_number": "25506883",\n    "items": "5.02",\n    "size": 263081,\n    "is_xbrl": true,\n  
"is_inline_xbrl": true,\n    "primary_document": "ef20040370_8k.htm",\n    "primary_doc_description": "FORM 8-K",\n
"source_file": "https://www.sec.gov/Archives/edgar/data/320193/000114036125000228/ef20040370_8k.htm"\n  }\n]',
                type='text'
            )
        ],
        is_error=False,
        tool_use_id='mcptoolu_01Xbudgikg27TqurBgRUM9sm',
        type='mcp_tool_result'
    ),
    BetaTextBlock(
        citations=None,
        text="Based on the search results, Apple's most recent 8-K filing was submitted on **May 12, 2025** (with a
report date of May 5, 2025). This filing covered items 8.01 and 9.01.\n\nLooking at Apple's recent 8-K filing 
history, they a

The model combines knowledge from **both** feeds and produces an answer grounded in the data they serve.

## 4. Continuing the same conversation
To keep context, simply append the assistant's previous reply & your new question to the `messages` list.  
We'll ask a follow-up that references the first answer.

In [ ]:
# Store the first turn so we can continue the thread
conversation = [
    {"role": "user", "content": "When did Apple last file an 8-K report?"},
    {"role": "assistant", "content": response.content}
]

# New user question, building on the prior response
conversation.append({
    "role": "user",
    "content": "Great, could you summarize the key points from that 8-K in bullet form?"
})

follow_up = client.beta.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=750,
    messages=conversation,
    mcp_servers=MCP_SERVERS,
    extra_headers={"anthropic-beta": "mcp-client-2025-04-04"}
)
print(follow_up.content)

[
    BetaTextBlock(
        citations=None,
        text="I'll extract and summarize the key points from Apple's most recent 8-K filing for you.",
        type='text'
    ),
    BetaMCPToolUseBlock(
        id='mcptoolu_011p2VoXx3FVZR4M84JTbnyY',
        input={'accession_number': '0001140361-25-018400', 'type': 'text'},
        name='Extract_and_classify_SEC_filing_content',
        server_name='mcp_sec_apibricks_io',
        type='mcp_tool_use'
    ),
    BetaMCPToolResultBlock(
        content=[
            BetaTextBlock(
                citations=None,
                text='{\n  "accession_number": "0001140361-25-018400",\n  "form_type": "8-K",\n  "items": [\n    
{\n      "item_number": "801",\n      "item_title": "Other Events",\n      "content": "Other Events. On May 12, 
2025, Apple Inc. (“Apple”) consummated the issuance and sale of $1,500,000,000 aggregate principal amount of its 
4.000% Notes due 2028 (the “2028 Notes”), $1,000,000,000 aggregate principal amount of its 4.200% Notes due 2030 
(the “2030 Notes”), $1,000,000,000 aggregate principal amount of its 4.500% Notes due 2032 (the “2032 Notes”) and 
$1,000,000,000 aggregate principal amount of its 4.750% Notes due 2035 (the “2035 Notes” and, together with the 
2028 Notes, the 2030 Notes, and the 2032 Notes, the “Notes”), pursuant to an underwriting agreement (the 
“Underwriting Agreement”) dated May 5, 2025 among Apple and Goldman Sachs & Co. LLC, Barclays Capital Inc., BofA 
Securities, Inc. and J.P. Morgan Securities LLC, as representatives of the several underwriters named therein. The 
Notes are being issued pursuant to an indenture, dated as of November 1, 2024 (the “Indenture”), between Apple and 
The Bank of New York Mellon Trust Company, N.A., as trustee, together with the officer’s certificate, dated May 12,
2025 (the “Officer’s Certificate”), issued pursuant to the Indenture establishing the terms of each series of 
Notes. The Notes are being issued pursuant to Apple’s Registration Statement on Form S-3 filed with the Securities 
and Exchange Commission and dated November 1, 2024 (Reg. No. 333-282937) (the “Registration Statement”). Interest 
on the 2028 Notes, the 2030 Notes, the 2032 Notes, and the 2035 Notes will be paid semi-annually in arrears on May 
12 and November 12 of each year, beginning on November 12 , 2025 . The 2028 Notes will mature on May 12, 2028 . The
2030 Notes will mature on May 12, 2030 . The 2032 Notes will mature on May 12, 2032. The 2035 Notes will mature on 
May 12, 2035 . The Notes will be Apple’s senior unsecured obligations and will rank equally with Apple’s other 
unsecured and unsubordinated debt from time to time outstanding. The foregoing description of the Notes and related
agreements is qualified in its entirety by the terms of the Underwriting Agreement, the Indenture and the Officer’s
Certificate (including the forms of the Notes). Apple is furnishing the Underwriting Agreement and the Officer’s 
Certificate (including the forms of the Notes) attached hereto as Exhibits 1.1 and 4.1 through 4.5, respectively, 
and they are incorporated herein by reference. The Indenture is filed as Exhibit 4.1 to the Registration Statement.
An opinion regarding the legality of the Notes is filed as Exhibit 5.1, and is incorporated by reference into the 
Registration Statement; and a consent relating to the incorporation of such opinion is incorporated by reference 
into the Registration Statement and is filed as Exhibit 23.1 by reference to its inclusion within Exhibit 5.1."\n  
},\n    {\n      "item_number": "901",\n      "item_title": "Financial Statements and Exhibits",\n      "content": 
"Item 9.01 Financial Statements and Exhibits. (d) Exhibits. Exhibit Number Exhibit Description 1.1 Underwriting 
Agreement, dated May 5, 2025, among Apple Inc. and Goldman Sachs & Co. LLC, Barclays Capital Inc., BofA Securities,
Inc. and J.P. Morgan Securities LLC, as representatives of the several underwriters named therein 4.1 Officer’s 
Cer

## 5. Mixing structured & natural-language follow-ups
Claude can answer highly structured questions too. Let's request **just the filing date and a JSON summary**:

In [ ]:
conversation.append({
    "role": "assistant",
    "content": follow_up.content
})

conversation.append({
    "role": "user",
    "content": "Return ONLY a JSON object with keys `filing_date` and `highlights`."
})

json_reply = client.beta.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=300,
    messages=conversation,
    mcp_servers=MCP_SERVERS,
    extra_headers={"anthropic-beta": "mcp-client-2025-04-04"}
)
print(json_reply.content)

[
    BetaMCPToolUseBlock(
        id='mcptoolu_014oXhZJz2QnU7wSDjT6g3CJ',
        input={'accession_number': '0001140361-25-018400', 'type': 'text'},
        name='Extract_and_classify_SEC_filing_content',
        server_name='mcp_sec_apibricks_io',
        type='mcp_tool_use'
    ),
    BetaMCPToolResultBlock(
        content=[
            BetaTextBlock(
                citations=None,
                text='{\n  "accession_number": "0001140361-25-018400",\n  "form_type": "8-K",\n  "items": [\n    
{\n      "item_number": "801",\n      "item_title": "Other Events",\n      "content": "Other Events. On May 12, 
2025, Apple Inc. (“Apple”) consummated the issuance and sale of $1,500,000,000 aggregate principal amount of its 
4.000% Notes due 2028 (the “2028 Notes”), $1,000,000,000 aggregate principal amount of its 4.200% Notes due 2030 
(the “2030 Notes”), $1,000,000,000 aggregate principal amount of its 4.500% Notes due 2032 (the “2032 Notes”) and 
$1,000,000,000 aggregate principal amount of its 4.750% Notes due 2035 (the “2035 Notes” and, together with the 
2028 Notes, the 2030 Notes, and the 2032 Notes, the “Notes”), pursuant to an underwriting agreement (the 
“Underwriting Agreement”) dated May 5, 2025 among Apple and Goldman Sachs & Co. LLC, Barclays Capital Inc., BofA 
Securities, Inc. and J.P. Morgan Securities LLC, as representatives of the several underwriters named therein. The 
Notes are being issued pursuant to an indenture, dated as of November 1, 2024 (the “Indenture”), between Apple and 
The Bank of New York Mellon Trust Company, N.A., as trustee, together with the officer’s certificate, dated May 12,
2025 (the “Officer’s Certificate”), issued pursuant to the Indenture establishing the terms of each series of 
Notes. The Notes are being issued pursuant to Apple’s Registration Statement on Form S-3 filed with the Securities 
and Exchange Commission and dated November 1, 2024 (Reg. No. 333-282937) (the “Registration Statement”). Interest 
on the 2028 Notes, the 2030 Notes, the 2032 Notes, and the 2035 Notes will be paid semi-annually in arrears on May 
12 and November 12 of each year, beginning on November 12 , 2025 . The 2028 Notes will mature on May 12, 2028 . The
2030 Notes will mature on May 12, 2030 . The 2032 Notes will mature on May 12, 2032. The 2035 Notes will mature on 
May 12, 2035 . The Notes will be Apple’s senior unsecured obligations and will rank equally with Apple’s other 
unsecured and unsubordinated debt from time to time outstanding. The foregoing description of the Notes and related
agreements is qualified in its entirety by the terms of the Underwriting Agreement, the Indenture and the Officer’s
Certificate (including the forms of the Notes). Apple is furnishing the Underwriting Agreement and the Officer’s 
Certificate (including the forms of the Notes) attached hereto as Exhibits 1.1 and 4.1 through 4.5, respectively, 
and they are incorporated herein by reference. The Indenture is filed as Exhibit 4.1 to the Registration Statement.
An opinion regarding the legality of the Notes is filed as Exhibit 5.1, and is incorporated by reference into the 
Registration Statement; and a consent relating to the incorporation of such opinion is incorporated by reference 
into the Registration Statement and is filed as Exhibit 23.1 by reference to its inclusion within Exhibit 5.1."\n  
},\n    {\n      "item_number": "901",\n      "item_title": "Financial Statements and Exhibits",\n      "content": 
"Item 9.01 Financial Statements and Exhibits. (d) Exhibits. Exhibit Number Exhibit Description 1.1 Underwriting 
Agreement, dated May 5, 2025, among Apple Inc. and Goldman Sachs & Co. LLC, Barclays Capital Inc., BofA Securities,
Inc. and J.P. Morgan Securities LLC, as representatives of the several underwriters named therein 4.1 Officer’s 
Certificate of Apple Inc., dated May 12, 2025 4.2 Form of Global Note representing the 2028 Notes (included in 
Exhibit 4.1) 4.3 Form of Global Note representing the 2030 Notes